In [42]:
"""
now trying to calculate route from first to last waypoint of each linepoint
"""
import uuid
import gpxpy
import gpxpy.gpx
import numpy as np
import osmnx as ox
import networkx as nx
from itertools import combinations
import shapely
import gpxpy.gpx
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

# Step 1: Create a Street Network Graph
place_name = "Lustenau, Vorarlberg, Austria"
G = ox.graph_from_place(place_name, network_type='drive', simplify=False)
print(len(G.edges))
G_proj = G.to_undirected()
# G_proj.add_edge(())
print(len(G_proj.edges))
stats = ox.basic_stats(G_proj)

lustenau_nodes_df, lustenau_edges_df = ox.graph_to_gdfs(G_proj)

11112
5753


In [43]:
lustenau_edges_df[lustenau_edges_df.name.str.contains('Müller', na=False)]

osmid  oneway lanes  ref          name  \
u          v           key                                               
310607039  7017399096  0    150250260   False   NaN  NaN  Müllerstraße   
           7017399092  0    150250262   False   NaN  NaN  Müllerstraße   
           7017399094  0    150250262   False   NaN  NaN  Müllerstraße   
347546106  7011583126  0    150250262   False   NaN  NaN  Müllerstraße   
1631767576 11346018106 0    150250260   False   NaN  NaN  Müllerstraße   
1631767608 11346018106 0    150250260   False   NaN  NaN  Müllerstraße   
           1631767619  0    150250260   False   NaN  NaN  Müllerstraße   
1631767619 1631767632  0    150250260   False   NaN  NaN  Müllerstraße   
1631767632 7017399096  0    150250260   False   NaN  NaN  Müllerstraße   
1631767690 7017399094  0    150250262   False   NaN  NaN  Müllerstraße   
           1862845120  0    150250262   False   NaN  NaN  Müllerstraße   
1631767722 1862845120  0    150250262   False   NaN  NaN  Müllerstraße   
7011583126 7017399092  0    150250262   False   NaN  NaN  Müllerstraße   

                                highway maxspeed  reversed  length bridge  \
u          v           key                                                  
310607039  7017399096  0    residential       40      True  71.424    NaN   
           7017399092  0    residential       40      True  17.165    NaN   
           7017399094  0    residential       40     False  13.023    NaN   
347546106  7011583126  0    residential       40     False  17.842    NaN   
1631767576 11346018106 0    residential       40     False   5.092    NaN   
1631767608 11346018106 0    residential       40      True  41.981    NaN   
           1631767619  0    residential       40     False   6.741    NaN   
1631767619 1631767632  0    residential       40     False  12.691    NaN   
1631767632 7017399096  0    residential       40     False  10.270    NaN   
1631767690 7017399094  0    residential       40      True  43.667    NaN   
           1862845120  0    residential       40     False  27.989    NaN   
1631767722 1862845120  0    residential       40      True  27.613    NaN   
7011583126 7017399092  0    residential       40     False  29.158    NaN   

                           junction width tunnel  \
u          v           key                         
310607039  7017399096  0        NaN   NaN    NaN   
           7017399092  0        NaN   NaN    NaN   
           7017399094  0        NaN   NaN    NaN   
347546106  7011583126  0        NaN   NaN    NaN   
1631767576 11346018106 0        NaN   NaN    NaN   
1631767608 11346018106 0        NaN   NaN    NaN   
           1631767619  0        NaN   NaN    NaN   
1631767619 1631767632  0        NaN   NaN    NaN   
1631767632 7017399096  0        NaN   NaN    NaN   
1631767690 7017399094  0        NaN   NaN    NaN   
           1862845120  0        NaN   NaN    NaN   
1631767722 1862845120  0        NaN   NaN    NaN   
7011583126 7017399092  0        NaN   NaN    NaN   

                                                                   geometry  
u          v           key                                                   
310607039  7017399096  0    LINESTRING (9.65505 47.42641, 9.65587 47.42608)  
           7017399092  0    LINESTRING (9.65505 47.42641, 9.65493 47.42628)  
           7017399094  0    LINESTRING (9.65505 47.42641, 9.65515 47.42650)  
347546106  7011583126  0    LINESTRING (9.65458 47.42592, 9.65471 47.42606)  
1631767576 11346018106 0    LINESTRING (9.65675 47.42573, 9.65669 47.42575)  
1631767608 11346018106 0    LINESTRING (9.65621 47.42594, 9.65669 47.42575)  
           1631767619  0    LINESTRING (9.65621 47.42594, 9.65613 47.42598)  
1631767619 1631767632  0    LINESTRING (9.65613 47.42598, 9.65599 47.42603)  
1631767632 7017399096  0    LINESTRING (9.65599 47.42603, 9.65587 47.42608)  
1631767690 7017399094  0    LINESTRING (9.65547 47.42683, 9.65515 47.42650)  
           1862845120  0    LINESTRING (9.65547 

In [44]:
%%capture
fig, ax = ox.plot_graph(G_proj.to_undirected(), node_size=1, bgcolor='k', node_color='w', edge_color='r', edge_linewidth=0.2, figsize=(100,100), dpi=600, save=True, filepath='tryout.png')
fig.savefig()

TypeError: Figure.savefig() missing 1 required positional argument: 'fname'

In [45]:
import networkx as nx

def create_eulerian_graph(G):
    # Find nodes with odd degree
    odd_degree_nodes = [v for v, d in G.degree() if d % 2 == 1]

    # Add edges to make the graph Eulerian
    # Note: This is a simplified approach; in practice, you'd want to add 
    # edges that minimally increase the total weight (distance) of the graph
    for i in range(0, len(odd_degree_nodes), 2):
        shortest_path = nx.shortest_path(G, odd_degree_nodes[i], odd_degree_nodes[i + 1], weight='weight')
        nx.add_path(G, shortest_path)

    return G

# Your original graph
G2 = G_proj

# Make the graph Eulerian
eulerian_G = create_eulerian_graph(G2)

# Find an Eulerian circuit
circuit = list(nx.eulerian_circuit(eulerian_G))

waypoints = []

for index, node in enumerate(circuit):
    if index == 0:
        continue
    node1 = circuit[index - 1][0]
    waypoints.append(gpxpy.gpx.GPXWaypoint(latitude=G2.nodes[node1]['x'], longitude=G2.nodes[node1]['y'], name=""))

gpx = gpxpy.gpx.GPX()

# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Create first segment in our GPX track:
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

for waypoint in waypoints:
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(waypoint.longitude, waypoint.latitude))

print(gpx.to_xml())



<?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="gpx.py -- https://github.com/tkrajina/gpxpy">
  <trk>
    <trkseg>
      <trkpt lat="47.3948065" lon="9.6866935">
      </trkpt>
      <trkpt lat="47.3954183" lon="9.6877153">
      </trkpt>
      <trkpt lat="47.3958669" lon="9.6885035">
      </trkpt>
      <trkpt lat="47.3963717" lon="9.6893882">
      </trkpt>
      <trkpt lat="47.3967984" lon="9.6901313">
      </trkpt>
      <trkpt lat="47.3974325" lon="9.6912349">
      </trkpt>
      <trkpt lat="47.3979951" lon="9.6921921">
      </trkpt>
      <trkpt lat="47.3981445" lon="9.692327">
      </trkpt>
      <trkpt lat="47.398209" lon="9.6923979">
      </trkpt>
      <trkpt lat="47.3982839" lon="9.6924538">
      </trkpt>
      <trkpt lat="47.3983697" lon="9.692492">
      </t